In [ ]:
## Mean518

In [1]:
import time
import datetime

In [2]:
start_ = 
end_ =
company =

start = int(time.mktime(datetime.datetime.strptime(start_, '%Y-%m-%d %H:%M:%S').timetuple()))*1000
end = int(time.mktime(datetime.datetime.strptime(end_, '%Y-%m-%d %H:%M:%S').timetuple()))*1000

In [3]:
import pymysql
import pandas as pd

db = pymysql.connect(host='', 
                         user='', password='', db='',
                         charset='utf8')
curs = db.cursor()
sql = """

"""
curs.execute(sql,(company,start,end))
data = list(curs.fetchall())
table = pd.DataFrame(columns=[])


for i in data :
    dict = { 
        'room_id': [i[0]],
        'type': ['채팅상담' if i[1]==1 else '전화상담'],
        'age': [str(2021-int(i[2])+1)[0]+'0대'], 
        'sex': ['여성' if i[3]=='F' else '남성'],
        'ten1': [i[4]],'ten2': [i[5]],'ten3': [i[6]],
        '연도': [int(str(i[7].strftime('%Y')))]   ,
        '월': [int(str(i[7].strftime('%m')))]   ,
        '시간대': [int(str(i[7].strftime('%H')))]   
    }
    
    table = table.append(pd.DataFrame(dict))

In [ ]:
## 동일인이 여러번 상담 받는 경우를 고려하여 sql 에서 group by 안해주고 밑에서 중복된 행 제거하였음
table = table.drop_duplicates()
table = table.sort_values(by=['type','월','시간대'])

## 엑셀로 저장
table.to_excel(f'{company}_{start_[0:7]}_{end_[0:7]}/{company}_{start_[0:7]}_{end_[0:7]}.xlsx', index=None)

In [12]:
import pyecharts
pyecharts.online()
from pyecharts import Line, Bar, Pie, Overlap
from collections import Counter

In [14]:
f_table = table[table['sex'] == '여성']
m_table = table[table['sex'] == '남성']

ff = []
mm = []
for i in list(set(table['age'].values)) :
    ff.append(len([ a for a in f_table['age'] if a == i] ))
    mm.append(len([ a for a in m_table['age'] if a == i] ))

pie = Pie('연령별비율')
pie.add("", list(set(table['age'].values)),ff, is_label_show=True, label_text_size=20, legend_text_size=15, legend_top='bottom',)
pie

In [15]:
pie = Pie('성별비율')
pie.add("",['여자','남자'],[len(f_table),len(m_table)], is_label_show=True, label_text_size=20, legend_text_size=15, legend_top='bottom')
pie

## 상담텍스트 분석

# 여자

In [ ]:
table[table['type']=='채팅상담']

room_id = list(f_table[f_table['type']=='채팅상담']['room_id'].values)

In [19]:
import pymysql
import pandas as pd
from Crypto.Cipher import AES
import base64
import time

In [20]:
customer2 = []
customer1 = []
id_cnt = 0
# data_df_1 만들기
for iii in room_id :
    try:
        id_cnt += 1
        print(f'id {id_cnt}번째 추출중')
        data_df_1 = pd.DataFrame(columns=[''])
        db = pymysql.connect()
        curs = db.cursor()
        sql = """
        
            """
        curs.execute(sql,iii)
        data = list(curs.fetchall())

        for i in range(len(data)) :
            code=data[i][3]
            obj = AES.new()
            unpadding = lambda s: s[:-ord(s[-1])]
            text = unpadding(obj.decrypt(base64.b64decode(code)).decode('utf-8'))
            dict = {'room_id':[data[i][0]], 'counselor_id':[data[i][1]], 'teller':[data[i][2]], 'text':[text], 'date':[data[i][4]] }
            data_df_1 = data_df_1.append(pd.DataFrame(dict))

            
        counselor_id = data_df_1['counselor_id'].iloc[0]
        customer = []
        text = ''
        for i in range(len(data_df_1)) :
            this_text = data_df_1.iloc[i]
            if this_text['teller'] != counselor_id : 
                customer.append(this_text['text'])

        customer1.extend(customer[6:])
        customer2.append(customer[6:])
        
    except : continue

id 1번째 추출중
id 2번째 추출중
id 3번째 추출중


In [23]:
from hanspell import spell_checker
from collections import Counter
from eunjeon import Mecab

data = ''
for i in customer1 :
    text = i
    result = spell_checker.check(text)
    text = result.as_dict()['checked']
    data += text+' '

In [24]:
tagger = Mecab() 
posed_data = tagger.pos(data)

data_nng_va = [ i[0] for i in posed_data if ('NNG' in i[1]) or ('VA' in i[1]) ]
count = Counter(data_nng_va)
most_word = count.most_common()

stop_words = '것 있 때 좋 없 많 그렇 정도 전 안녕 부분 그때 곳 이번 당시 넵 점 원인 순간 얼마 안 낮 저녁 주 예약 신청 대면 주지 현상 경우 관리 방식 기존 특정 다르 그런가요 사이 늦 음 같 하 정도 괜찮 날 '
stop_words = stop_words.split(' ')

most_word = [i for i in most_word if not i[0] in stop_words and int(i[1]) >= 2 ]
most_word_dict = {i[0]:i[1] for i in most_word}

In [25]:
bar = Bar("핵심어")
bar.add("여자", [i[0] for i in most_word[:8]], [i[1] for i in most_word[:8]], label_text_size=18,
        is_label_show=True,  legend_text_size=15)

bar

In [26]:
most_word_table = pd.DataFrame(list(most_word_dict.items()),
                           columns=['핵심어', '사용빈도']).iloc[:15]

In [27]:
relate_word = []
for word in most_word_table['핵심어'] :
    one = ''
    for i in customer1 :
        if word in i :
            one += i+' '

    posed_data = tagger.pos(one)
    data_nng_va = [ i[0] for i in posed_data if ('NNG' in i[1]) or ('VA' in i[1]) ]
    count = Counter(data_nng_va)
    most_word = count.most_common()
    stop_words = '것 있 때 좋 없 많 그렇 정도 전 안녕 부분 그때 곳 이번 당시 넵 점 원인 순간 얼마 안 낮 저녁 주 예약 신청 대면 주지 현상 경우 관리 방식 기존 특정 다르 그런가요 사이 늦 음 같 하 정도'
    stop_words = stop_words.split(' ')
    most_word = [i for i in most_word if not i[0] in stop_words and int(i[1]) >= 1 ]

    a = most_word[2:6]
    relate_word.append(a)

    
most_word_table['연관어'] = relate_word
most_word_table.to_excel(f'{company}_{start_[0:7]}_{end_[0:7]}/핵심어/여자_{len(room_id)}명.xlsx', index=None)

## 정서강도

In [1]:
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()
model.load_state_dict(torch.load('C:/Users/lll/3.rivised_eap/movie_review_model_epoch_15.h5'))

<All keys matched successfully>

In [2]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 2 GPU(s) available.
We will use the GPU: GeForce RTX 3080


In [3]:
# 입력 데이터 변환
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # 어텐션 마스크 초기화
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids).to(device).long()
    masks = torch.tensor(attention_masks).to(device).long()

    return inputs, masks

In [4]:
# 문장 테스트
def text_score(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device).long()
    b_input_mask = masks.to(device).long()
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    score = outputs[0]
    score = score.detach().cpu().numpy()

    return score

In [5]:
def result(score) :
    if np.argmax(score) == 0 :
        return print(f'부정감  {score[0][0]}')
    if np.argmax(score) == 1 :
        return print(f'긍정감 {score[0][1]}')

In [9]:
scores = text_score(['그때 정말 너무 힘들어서 우울증 걸렸었어요'])
print(scores)
print(result(scores))

[[ 4.1928153 -3.8970366]]
부정감  4.19281530380249
None


In [10]:
scores = text_score(['너무 안좋은 기억이라 떠올리고 싶지도 않아요'])
print(scores)
print(result(scores))

[[ 4.207776  -3.8955102]]
부정감  4.207776069641113
None


In [11]:
scores = text_score(['너무 피곤해서 좀 쉬고싶달까.. 번아웃된 느낌이었어요'])
print(scores)
print(result(scores))

[[ 4.2215576 -3.9119241]]
부정감  4.2215576171875
None


In [12]:
scores = text_score(['제가 해냈다는게 정말 믿기지 않고, 스스로 자랑스러웠어요'])
print(scores)
print(result(scores))

[[-4.2507052  4.0970736]]
긍정감 4.097073554992676
None


In [13]:
scores = text_score(['뿌듯하기도 하고, 그동안 고생한게 생각이 나서 눈물이 났어요'])
print(scores)
print(result(scores))

[[-4.2556186  4.0912905]]
긍정감 4.091290473937988
None


In [14]:
scores = text_score(['고마운 사람들이 곁에 많았죠'])
print(scores)
print(result(scores))

[[-4.236262   4.0678177]]
긍정감 4.067817687988281
None


In [57]:
chatting = table[table['type']=='채팅상담']
room_id = list(chatting['room_id'].values)

In [59]:
customer2 = []
customer1 = []
id_cnt = 0

for iii in room_id :
    try:
        id_cnt += 1
        print(f'id {id_cnt}번째 추출중')
        data_df_1 = pd.DataFrame(columns=[''])
        db = pymysql.connect('')
        curs = db.cursor()
        sql = """
        select 
            """
        curs.execute(sql,iii)
        data = list(curs.fetchall())

        for i in range(len(data)) :
            code=data[i][3]
            obj = AES.new()
            unpadding = lambda s: s[:-ord(s[-1])]
            text = unpadding(obj.decrypt(base64.b64decode(code)).decode('utf-8'))
            dict = {'':[data[i][0]], '':[data[i][1]], '':[data[i][2]], '':[text], '':[data[i][4]] }
            data_df_1 = data_df_1.append(pd.DataFrame(dict))

            
        counselor_id = data_df_1[''].iloc[0]
        customer = []
        text = ''
        for i in range(len(data_df_1)) :
            this_text = data_df_1.iloc[i]
            if this_text['teller'] != counselor_id : 
                customer.append(this_text['text'])

        customer1.extend(customer[6:])
        customer2.append(customer[6:])
        
    except : continue

id 1번째 추출중
id 2번째 추출중
id 3번째 추출중
id 4번째 추출중
id 5번째 추출중


In [60]:
bert_binary = []
for person in customer2 :
    box = []
    for i in person :
        logits = text_score([i])
        box.append(list(logits[0]))
    bert_binary.append(box)

In [64]:
def make_line(number) :
    neg = []
    pos = []
    for i in bert_binary[number] :
        neg.append(i[0]+1)
        pos.append(i[1]-1)

    line = Line("상담진행에 따른 감정 변화")
    line.add("부정적 감정", list(range(len(neg))), neg , mark_line=["max", "average"] )
    # line.add("pos", list(range(len(pos))), pos )
    return line

In [66]:
make_line(2)

In [70]:
make_line(0)

In [65]:
make_line(1)

In [68]:
make_line(4)

In [54]:
neg = []
pos = []
for i in bert_binary[1] :
    neg.append(i[0]+1)
    pos.append(i[1]-1)

line = Line("상담진행에 따른 감정 변화")
line.add("부정적 감정", list(range(len(neg))), neg , mark_line=["max", "average"])
# line.add("pos", list(range(len(pos))), pos )
line

In [203]:
[i[0] for i in bert_binary[0]]

[-0.7851768,
 0.0015542323,
 -0.10825307,
 2.182775,
 -3.0398948,
 -1.3183849,
 -1.5005352,
 0.69489044,
 -0.8953393,
 -2.786751,
 -1.9802078,
 1.121151,
 0.034409396,
 3.4047282,
 -1.7567788,
 1.6721853,
 -0.03218369,
 -2.3229449,
 -0.028219443,
 -0.028219443,
 -1.9399623,
 0.34173,
 -1.6298008,
 0.28298745,
 -0.59399503,
 -0.60481447,
 -0.2244518,
 0.18701635,
 -1.1951472,
 -0.3822032,
 -1.7605773,
 -0.8217027,
 -2.3575628,
 0.8793114,
 1.2798269,
 -2.373863,
 -1.8895856,
 1.4890606,
 -1.5953443,
 -1.5645463,
 -1.2404215,
 -0.79700625,
 -0.7631922,
 1.1188234,
 -1.0939779,
 -2.1343167,
 -1.1062902,
 0.23519306,
 -2.785462,
 -2.263027]

In [98]:
df = pd.DataFrame([i[0]+1 for i in bert_binary[0]], columns=['customer0'])
for n in range(1,len(bert_binary)) :
    df = df.join(pd.Series([i[0]+1 for i in bert_binary[n]]).rename(f'customer{n}'), how='outer')

df.to_excel(f'{company}_{start_[0:7]}_{end_[0:7]}/_정서강도/상담진행따른_우울감.xlsx', index=None)

In [ ]:
df = pd.DataFrame([i[1]-1 for i in bert_binary[0]], columns=['customer0'])
for n in range(1,len(bert_binary)) :
    df = df.join(pd.Series([i[1]-1 for i in bert_binary[n]]).rename(f'customer{n}'), how='outer')

df.to_excel(f'{company}_{start_[0:7]}_{end_[0:7]}/_정서강도/상담진행따른_긍정감.xlsx', index=None)